Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 


# imports

In [3]:
!pip install transformers
!pip install catboost
!pip install detoxify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 106.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully unin

In [5]:
import pandas as pd 
import numpy as np 

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import RandomizedSearchCV

import torch
import transformers
from tqdm import notebook
from torch import tensor

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re
from sklearn.utils import shuffle

from catboost import Pool, CatBoostClassifier

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [6]:
DATA_PATH = 'toxic_comments.csv'
#DATA_PATH = '/datasets/toxic_comments.csv'

In [7]:
# напишем функцию для очистки и лемматизации текста с маркированными частями речи
def clear_text(text):
  #очистка текста 
  return ' '.join(re.sub(r'[^a-zA-Z\' ]', ' ', text).split())

In [8]:
#проверка на GPU, если есть, то ускорит BERT
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
!nvidia-smi

Wed Mar 22 18:01:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    29W /  70W |      3MiB / 15360MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# first look 

In [10]:
df = pd.read_csv(DATA_PATH)
initial_df = df.copy()

In [11]:
df.sample(10)

,Unnamed: 0,text,toxic
99556,99652,""" May 2009 (UTC)\n\nDo you mean you can read a...",0
77105,77181,you're wasting your time biatch you know i'll ...,1
69120,69188,"Seriously, please stop that. Do not continue t...",0
38446,38492,", 24 April 2014 (UTC)|decline=I might have acc...",0
72570,72641,"""\n\nWell it has been changed, also it may hav...",0
122871,122977,"""\nI enjoyed reading that proposal. It gave m...",0
148038,148194,"""::Exactly. And it's about damn time someone a...",0
153849,154006,Who are you to say which source is reliable? Y...,0
120283,120388,""":::::It is not a question of whether Soriano ...",0
42644,42694,"""\nActually, quite a lot of religions start wi...",0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


# data preprocessing

In [13]:
df = df.drop(['Unnamed: 0'], axis = 1) 

In [14]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [15]:
df.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

таргет сильно несбалансирован, лучше сделать downsampling, так модель луше выхватит целевой признак, заодно и сэко

In [17]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [18]:
df['text'] = df['text'].apply(lambda x: clear_text(x))

In [19]:
df['text'] = df['text'].str.lower()

# catboost baseline

In [20]:
df

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww he matches this background colour i'm se...,0
2,hey man i'm really not trying to edit war it's...,0
3,more i can't make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159287,and for the second time of asking when your vi...,0
159288,you should be ashamed of yourself that is a ho...,0
159289,spitzer umm theres no actual article for prost...,0
159290,and it looks like it was actually you who put ...,0


In [21]:
# разделим выборки на обучающую, валидационную и тестовую в пропорции 60%-20%-20%
target = df['toxic']
features = df[['text']]

#указываем stratify=target, чтобы сохранить соотношение классов при разделении
#сначала разделим на обучающую и тестовую в соотношении 60%-40%
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.4, random_state=42, stratify=target) 

#разделим тестовую выборку пополам на тестовую и валидационную
x_valid, x_test, y_valid, y_test = train_test_split(
    x_test, y_test, test_size=0.50, random_state=42)

In [71]:
model_ctb_baseline = CatBoostClassifier(text_features = ['text'], 
                           random_seed = 42, 
                           eval_metric= 'F1',
                           task_type= 'GPU', 
                           verbose = False)
model_ctb_baseline.fit(x_train, y_train)

In [72]:
predicted_valid = model_ctb_baseline.predict(x_valid)
print("F1_score:", f1_score(y_valid, predicted_valid))

F1_score: 0.7617065556711758


получается, что катбуст из коробки уже показывает нужный результат, хорошая штука. 

# BERT

In [25]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
                                                    transformers.DistilBertTokenizer,
                                                    'distilbert-base-uncased')

# model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, 
#                                                     transformers.BertTokenizer, 
#                                                     'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
tokenized= df['text'].apply(lambda x: \
                             tokenizer.encode(x, add_special_tokens=True, 
                                              truncation=True, max_length=512))

In [27]:
tokenized[x_train.index]

155914    [101, 2265, 2033, 1037, 2512, 13433, 2615, 312...
155946    [101, 3374, 4087, 3291, 2007, 1996, 2682, 2930...
91674     [101, 8945, 3022, 1055, 3431, 1999, 20445, 243...
48586     [101, 2417, 7442, 6593, 2831, 7201, 1997, 1057...
34626     [101, 1045, 2572, 2047, 2000, 2023, 2609, 1998...
                                ...                        
145508    [101, 2092, 2061, 2521, 2045, 2003, 2053, 2107...
31323     [101, 11396, 2017, 4550, 2039, 2044, 2033, 204...
85790     [101, 3160, 2009, 3849, 2008, 12997, 2038, 204...
54711     [101, 5862, 16012, 7583, 2153, 2106, 2017, 319...
67881     [101, 1045, 2318, 1037, 6594, 11689, 7632, 428...
Name: text, Length: 95575, dtype: object

In [28]:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

In [29]:
padded_train = np.array([i + [0]*(max_len - len(i)) for i in tokenized[x_train.index].values])
padded_valid = np.array([i + [0]*(max_len - len(i)) for i in tokenized[x_valid.index].values])
padded_test = np.array([i + [0]*(max_len - len(i)) for i in tokenized[x_test.index].values])

attention_mask_train = np.where(padded_train != 0, 1, 0)
attention_mask_valid = np.where(padded_valid != 0, 1, 0)
attention_mask_test = np.where(padded_test != 0, 1, 0)


In [30]:
#перенесем все на GPU
masks = [attention_mask_train, attention_mask_valid, attention_mask_test]
padded_list = [padded_train, padded_valid, padded_test]

padded_cuda = []
masks_cuda = []

for padded in padded_list:
  padded_cuda.append(torch.LongTensor(padded).to(DEVICE))

for am in masks:
  masks_cuda.append(torch.LongTensor(am).to(DEVICE))

model.to(DEVICE)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [31]:
batch_size = 100
emdeddings_train = []
emdeddings_valid = []

padded_train = padded_cuda[0]
padded__valid = padded_cuda[1]
attention_mask_train = masks_cuda[0]
attention_mask_valid = masks_cuda[1]

#эмбединги на трейна 
for i in notebook.tqdm(range((padded_train.shape[0] // batch_size)+1)):
    batch = padded_train[batch_size*i:batch_size*(i+1)]
    attention_mask_batch = attention_mask_train[batch_size*i:batch_size*(i+1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    emdeddings_train.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/956 [00:00<?, ?it/s]

In [32]:
#эмбединги на трейна 
for i in notebook.tqdm(range((padded_cuda[1].shape[0] // batch_size)+1)):
    batch = padded_cuda[1][batch_size*i:batch_size*(i+1)]
    attention_mask_batch = masks_cuda[1][batch_size*i:batch_size*(i+1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    emdeddings_valid.append(batch_embeddings[0][:,0,:].cpu().numpy())


  0%|          | 0/319 [00:00<?, ?it/s]

In [33]:
features_train = np.concatenate(emdeddings_train)
features_valid = np.concatenate(emdeddings_valid)

## classification 

## LR 

In [35]:
lr = LogisticRegression(class_weight = 'balanced', random_state = 42)
lr.fit(features_train, y_train)
pred = lr.predict(features_valid)
print('f1 score on validation', f1_score(pred, y_valid)) 

f1 score on validation 0.6610926583438322


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


результат на эмбедингах не очень удовлетворительный

## ctb

In [36]:
%%time 

X_train_ctb = features_train.copy()

y_train_ctb = y_train.copy()

#без перебора гиперпаратметров, чтобы просто сэкономить ресурсы/время 
params = {'random_seed': 42,
          'eval_metric': 'F1',
          'task_type': 'GPU', 'verbose':False}

model = CatBoostClassifier(**params)

model.fit(X_train_ctb, y_train_ctb)

CPU times: user 34.7 s, sys: 13.2 s, total: 47.9 s
Wall time: 29 s


In [41]:
pred = model.predict(features_valid)
print('f1 score on validation', f1_score(pred, y_valid)) 

f1 score on validation 0.7047787610619469


 Можно было попробовать специализированный БЕРТ: https://huggingface.co/unitary/toxic-bert


у кэтбуста результат чуть получше, но все еще хуже чем результат "из коробки" на tfidf

## best model test

In [73]:
predicted_valid = model_ctb_baseline.predict(x_test)
print("F1_score:", f1_score(y_test, predicted_valid))

F1_score: 0.7777010700724888


#Итог

Лучше себя показала модель кэтбуста: быстро, просто, дешево, не требовалось никаких ухищрений, хватило внутренней магии + tfidf. 

F1 мера около 0.77, что не так уж плохо. 

Что еще можно сделать: 
  - поиграться с трешхолдом 
  - апсемплинг 
  - поперебирать еще параметры катбуста 
  - добавить другой бустинг 
  - посмотреть другие модели для эмбедингов
  - увеличить объем данных и вычислительную мощность, чтобы дообучить берт на этом датасете